In [7]:
class Node():
    """A node class for A* Pathfinding and Gbfs"""

    def __init__(self, parent=None, position=None):
        self.parent = parent
        self.position = position
        self.g = 0
        self.h = 0
        self.f = 0

    def __eq__(self, other):
        return self.position == other.position
        
def heuristic(a, b):
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

def translatePath(maze):
#switch every 'W' to 1 and everythng else to 0
    import copy
    newMaze = copy.deepcopy(maze)
    for i in range(0,len(newMaze)):
        for j in range(0,len(newMaze[i])):
            if newMaze[i][j]=='W':
                newMaze[i][j]=1
            else:
                newMaze[i][j]=0
    return newMaze

def getA(path):
    for i in range(0,len(path)):
        for j in range(0,len(path[i])):
            if path[i][j]=='A':
                return (i,j)
                
def getB(path):
    for i in range(0,len(path)):
        for j in range(0,len(path[i])):
            if path[i][j]=='B':
                return (i,j)

def transPathToInstractions(path):
    import operator
    st=""
    for i in range(0,len(path)-1):
        if tuple(map(operator.sub, path[i+1], path[i]))==(1,0):
            st=st+"U-"
        elif tuple(map(operator.sub, path[i+1], path[i]))==(-1,0):
            st=st+"D-"
        elif tuple(map(operator.sub, path[i+1], path[i]))==(0,1):
            st=st+"L-"
        else:
            st=st+"R-"
    st=st[:-1]
    #reverse the string
    st=st[::-1]
    st =st + " "
    return st

def calcPathCost(route,path,cost):
    price=0
    for i in range(1,len(route)-1):
        price=price+cost[path[route[i][0]][route[i][1]]]
    return price

def algoAstar(path,cost):
#export to output.txt
    with open('output.txt', 'w') as fd:
        fd.write(astar(path,cost,getA(path),getB(path)))

def algoGbfs(path,cost):
#export to output.txt
    with open('output.txt', 'w') as fd:
        fd.write(gbfs(path,cost,getA(path),getB(path)))

#greedy best first search
def gbfs(path,cost, start, end):
    maze=translatePath(path)
    # Create start and end node
    start_node = Node(None, start)
    start_node.g = start_node.h = start_node.f = 0
    end_node = Node(None, end)
    end_node.g = end_node.h = end_node.f = 0

    # Initialize both open and closed list
    open_list = []
    closed_list = []

    # Add the start node
    open_list.append(start_node)

    # Loop until you find the end
    while len(open_list) > 0:

        # Get the current node
        current_node = open_list[0]
        current_index = 0
        for index, item in enumerate(open_list):
            if item.f < current_node.f:
                current_node = item
                current_index = index
        # Pop current off open list, add to closed list
        open_list.pop(current_index)
        closed_list.append(current_node)

        # Found the goal
        if current_node == end_node:
            route = []
            current = current_node
            while current is not None:
                route.append(current.position)
                current = current.parent
            return transPathToInstractions(route)+str(calcPathCost(route,path,cost)) # Return reversed path

        # Generate children
        children = []
        for new_position in [(0, -1), (0, 1), (-1, 0), (1, 0)]: # Adjacent squares
            # Get node position
            node_position = (current_node.position[0] + new_position[0], current_node.position[1] + new_position[1])
            # Make sure within range
            if node_position[0] > (len(maze) - 1) or node_position[0] < 0 or node_position[1] > (len(maze[len(maze)-1]) -1) or node_position[1] < 0:
                continue
            # Make sure walkable terrain
            if maze[node_position[0]][node_position[1]] != 0:
                continue
            # Create new node
            new_node = Node(current_node, node_position)
            # Append
            children.append(new_node)
        # Loop through children
        for child in children:
            closed=False
            # Child is on the closed list
            for closed_child in closed_list:
                if child == closed_child:
                    closed=True
                    continue
            if(closed):
                continue
            # Create the f, g, and h values
            child.g = cost
            child.h = heuristic(child.position, end)
            child.f = child.h
            # Add the child to the open list
            open_list.append(child)
            open_list.sort(key=lambda x: x.h)
    return "no path"

#A* search
def astar(path,cost, start, end):
    maze=translatePath(path)
    # Create start and end node
    start_node = Node(None, start)
    start_node.g = start_node.h = start_node.f = 0
    end_node = Node(None, end)
    end_node.g = end_node.h = end_node.f = 0

    # Initialize both open and closed list
    open_list = []
    closed_list = []

    # Add the start node
    open_list.append(start_node)

    # Loop until you find the end
    while len(open_list) > 0:
        # Get the current node
        current_node = open_list[0]
        current_index = 0
        for index, item in enumerate(open_list):
            if item.f < current_node.f:
                current_node = item
                current_index = index
        # Pop current off open list, add to closed list
        open_list.pop(current_index)
        closed_list.append(current_node)
        # Found the goal
        if current_node == end_node:
            path = []
            current = current_node
            while current is not None:
                path.append(current.position)
                current = current.parent
            return transPathToInstractions(path)+str(current_node.h) # Return reversed path
        # Generate children
        children = []
        for new_position in [(0, -1), (0, 1), (-1, 0), (1, 0)]: # Adjacent squares
            # Get node position
            node_position = (current_node.position[0] + new_position[0], current_node.position[1] + new_position[1])
            # Make sure within range
            if node_position[0] > (len(maze) - 1) or node_position[0] < 0 or node_position[1] > (len(maze[len(maze)-1]) -1) or node_position[1] < 0:
                continue
            # Make sure walkable terrain
            if maze[node_position[0]][node_position[1]] != 0:
                continue
            # Create new node
            new_node = Node(current_node, node_position)
            # Append
            children.append(new_node)
        # Loop through children
        for child in children:
            closed=False
            # Child is on the closed list
            for closed_child in closed_list:
                if child == closed_child:
                    closed=True
                    continue      
            if(closed):
                continue
            # Create the f, g, and h values
            child.g = heuristic(child.position, end) 
            child.h = current_node.h + cost[path[child.position[0]][child.position[1]]]
            child.f = child.g + child.h
            # Child is already in the open list
            for open_node in open_list:
                if child == open_node and child.h > open_node.h:
                    continue    
            # Add the child to the open list
            open_list.append(child)
    return "no path"


In [8]:
cost={'A':0,'B':0,'R':1,'D':4,'W':None}
input = []
with open('input.txt', "r") as fd:
    input = fd.read().splitlines()
algo=input[0]
size=input[1]
#str to char array
path=[]
for i in range(2,len(input)):
    path=path+[list(input[i])]
path
if(algo=='A*'):
    algoAstar(path,cost)
elif(algo=='greedyBestFirst'):
    algoGbfs(path,cost)
else:
    print("invalid algo")